In [3]:
import pandas as pd

### Code which puts survey into flat format

In [4]:
#dairy flat file
dff = pd.DataFrame()

### ID mappings

In [ ]:
import toml
import os

print("Current working directory:", os.getcwd())


path = os.path.join('config', 'fertilizers.toml')
fert = toml.load(path)['fertilizers']

print(fert)

for i in fert:
    print(i)

Current working directory: c:\Users\steffan.thomas\Documents\CFT_api
{'fertilizer': [{'ID': 1, 'CFT_Name': 'Grass fresh managed - on farm', 'Certified': False, 'Regions': 'Global (world)', 'ofi_name': 'grass_fresh_managed'}, {'ID': 8, 'CFT_Name': 'Maize silage', 'Certified': False, 'Regions': 'Global (world)', 'ofi_name': 'maize_silage'}, {'ID': 6, 'CFT_Name': 'Grass silage - on farm', 'Certified': False, 'Regions': 'Global (world)', 'ofi_name': 'grass_silage'}, {'ID': 9, 'CFT_Name': 'Lucerne / Alfalfa silage', 'Certified': False, 'Regions': 'Global (world)', 'ofi_name': 'lucerne_alfalfa_silage'}, {'ID': 4, 'CFT_Name': 'Grass hay - on farm', 'Certified': False, 'Regions': 'Global (world)', 'ofi_name': 'grass_hay'}, {'ID': 11, 'CFT_Name': 'Straw', 'Certified': False, 'Regions': 'Global (world)', 'ofi_name': 'straw'}, {'ID': 13, 'CFT_Name': 'Wheat', 'Certified': False, 'Regions': 'Poland', 'ofi_name': 'wheat'}, {'ID': 20, 'CFT_Name': 'Other cereals', 'Certified': False, 'Regions': 'Europ

In [ ]:
import tomllib

with open("feed_items.toml", "rb") as f:
    cfg = tomllib.load(f)

FEED_ITEMS = {item["ofi_name"]: item for item in cfg["feed"]}
ID_MAP = {item["id"]: item["ofi_name"] for item in cfg["feed"]}

print(FEED_ITEMS)


In [ ]:
id_mappings = {
    "grazing_quality": {
        "HIGH": 1,
        "LOW": 2
    }
}

### Make checks to see if inputs match options


In [ ]:
units = {
    "fertilizer_application_rate": 12, # t/ha
    "feed_weight": 9, # t
}

# make dry matter conversions

In [ ]:
import toml

feed_items = toml.load("feed_items.toml")["feed_item"]
herd_sections = toml.load("herd_sections.toml")["herd_sections"]

feed_components_input = []

for feed in feed_items:
    ofi = feed["ofi_name"]
    item = feed["item"]
    region = feed["region"]

    for hs in herd_sections:
        feed_components_input.append({
            "item": item,
            "region": region,
            "herd_section": hs,
            "dry_matter": {
                "value": dff[f"feed.{ofi}.{hs}.tonne"],
                "unit": units["feed_weight"]
            },
            "certified": False
        })


### Code which loads flat table into JSON format

In [ ]:


farm_input = {
    "country": "Poland", 
    "territory": None,
    "climate": "Cool Temperate Moist",
    "average_temperature": {"value": 10, "unit": "°C"},
    "latitude": 52.679,
    "longitude": 20.030,
    "soil_characteristics": "Sandy Soils",
    "farm_identifier": dff["farm_id"]
}

general_input = {
    "grazing_area": {"value": dff["general.grazing_area"], "unit": "ha"},
    "feed_approach": 1, # 1 = dmi
    "fertilizer_approach": 2, # 2 = Grazing, grass silage and hay area combined
}

milk_production_input = {
    "variety": dff["main_breed_variety"],
    "reporting_year": dff["milk_year"],
    "date_time": "start",
    "date_month": 1, # Season always starts in Jan
    "name": dff["farm_id"] + str("_") + dff["milk_year"].astype(str),
    "product_dry": {"value": dff["total_milk_production_litres"], "unit": "litres"},
    "fat_content": dff["milk_fat_content_percent"],
    "protein_content": dff["milk_protein_content_percent"],
    "protein_measure": 1 # 1 = true protein
}

herd_sections_input = [
    {
        "phase": "calf_dairy",
        "animals": dff["calf_dairy.herd_count"],
        "live_weight": {
            "value": dff["calf_dairy.herd_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["calf_dairy.sold_count"],
        "sold_weight": {
            "value": dff["calf_dairy.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["calf_dairy.purchased_count"],
        "purchased_weight": {
            "value": dff["calf_dairy.purchased_weight_kg"],
            "unit": "kg"
        }
    },
    {
        "phase": "heifer",
        "animals": dff["heifer.herd_count"],
        "live_weight": {
            "value": dff["heifer.herd_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["heifer.sold_count"],
        "sold_weight": {
            "value": dff["heifer.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["heifer.purchased_count"],
        "purchased_weight": {
            "value": dff["heifer.purchased_weight_kg"],
            "unit": "kg"
        }
    },
    {
        "phase": "cow_milk",
        "animals": dff["cow_milk.herd_count"],
        "live_weight": {
            "value": dff["cow_milk.herd_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["cow_milk.sold_count"],
        "sold_weight": {
            "value": dff["cow_milk.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["cow_milk.purchased_count"],
        "purchased_weight": {
            "value": dff["cow_milk.purchased_weight_kg"],
            "unit": "kg"
        }
    },
    {
        "phase": "cow_dry",
        "animals": dff["cow_dry.herd_count"],
        "live_weight": {
            "value": dff["cow_dry.live_weight_kg"],
            "unit": "kg"
        },
        "sold_animals": dff["cow_dry.sold_count"],
        "sold_weight": {
            "value": dff["cow_dry.sold_weight_kg"],
            "unit": "kg"
        },
        "purchased_animals": dff["cow_dry.purchased_count"],
        "purchased_weight": {
            "value": dff["cow_dry.purchased_weight_kg"],
            "unit": "kg"
        }
    }
]

grazing_input = [
    {
        "herd_section": "calf_dairy",
        "days": dff["calf_dairy.grazing_days"],
        "hours": dff["calf_dairy.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["calf_dairy.grazing_quality"]] # has to be int  1 =  high, 2 = low
    },
    {
        "herd_section": "heifer",
        "days": dff["heifer.grazing_days"],
        "hours": dff["heifer.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["heifer.grazing_quality"]] # has to be int  1 =  high, 2 = low
    },
    {
        "herd_section": "cow_milk",
        "days": dff["cow_milk.grazing_days"],
        "hours": dff["cow_milk.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["cow_milk.grazing_quality"]] # has to be int  1 =  high, 2 = low
    },
    {
        "herd_section": "cow_dry",
        "days": dff["cow_dry.grazing_days"],
        "hours": dff["cow_dry.grazing_hours_per_day"],
        "category": 2, # 2 = Confined pasture 
        "quality": id_mappings["grazing_quality"][dff["cow_dry.grazing_quality"]] # has to be int  1 =  high, 2 = low
    }
]

fertilizers_input = [
    # 1. Cattle Slurry
    {
        "type": 32, # 32 = Cattle slurry - 0.26% N
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.cattle_slurry.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 2. Cattle Manure
    {
        "type": 26, # 26 = Cattle manure - 0.6% N
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.cattle_manure.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 3. Custom NPK
    {
        "type": 44, # 44 = Compose your own NPK
        "production": 8, # 8 = Europe 2014
        "custom_ingredients": {
            "n_total_percentage": 6, # 6% as defined by Pawel
            "n_ammonia_percentage": 6, # 6% as defined by Pawel
            "p2o5_percentage": 20, # 20% as defined by Pawel
            "p2o5_percentage_type_id": 4, # 4 = P2O5
            "k2o_percentage": 30, # 30% as defined by Pawel
            "k2o_percentage_type_id": 5 # 5 = K2O 
        },
        "application_rate": {
            "value": dff["fertilizers.custom_npk.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 4. Ammonium nitrate - 33.5% N (granulated)
    {
        "type": 46, # 46 = Ammonium nitrate - 33.5% N (granulated)
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.ammonium_nitrate.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 2 # 2 = inhibition
    },
    # 5. Urea ammonium nitrate solution - 32% N
    {
        "type": 22, # 22 = Urea ammonium nitrate solution - 32% N
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.urea_ammonium_nitrate.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 6. Urea - 46% N
    {
        "type": 45, # 45 = Urea - 46% N
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.urea.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 7. Calcium nitrate - 15.5% N
    {
        "type": 7, # 7 = Calcium nitrate - 15.5% N
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.calcium_nitrate.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 8. Ammonium sulphate nitrate - 26%N
    {
        "type": 4, # 4 = Ammonium sulphate nitrate - 26%N
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.ammonium_sulphate_nitrate.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 9. Limestone - 55% CaCO3 / 29%CaO
    {
        "type": 13, # 13 = Limestone - 55% CaCO3 / 29%CaO
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.limestone.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    },
    # 10. Muriate of potash / Potassium chloride - 60% K2O
    {
        "type": 16, # 16 = Muriate of potash / Potassium chloride - 60% K2O
        "production": 8, # 8 = Europe 2014
        "application_rate": {
            "value": dff["fertilizers.muriate_of_potash.t_per_ha"],
            "unit": units["fertilizer_application_rate"]
        },
        "application_date": "unknown",
        "rate_measure": "product",
        "inhibition": 1 # 1 = no inhibition
    }
]

feed_components_input = [
    # ---- We use anyof option 3 - Feed Component WITH Herd Section + Dry Matter

    # 1. Grass fresh managed - on farm
    {        
        "item": 1, # 1 = Grass fresh managed
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.grass_fresh_managed.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 1, # 1 = Grass fresh managed
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.grass_fresh_managed.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 1, # 1 = Grass fresh managed
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.grass_fresh_managed.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 1, # 1 = Grass fresh managed
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.grass_fresh_managed.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 8. Maize silage
    {        
        "item": 8, # 8 = Maize silage
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.maize_silage.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 8, # 8 = Maize silage
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.maize_silage.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 8, # 8 = Maize silage
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.maize_silage.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 8, # 8 = Maize silage
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.maize_silage.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 6. Grass silage - on farm
    {
        "item": 6, # 6 = Grass silage
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.grass_silage.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 6, # 6 = Grass silage
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.grass_silage.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 6, # 6 = Grass silage
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.grass_silage.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 6, # 6 = Grass silage
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.grass_silage.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 9. Lucerne / Alfalfa silage
    {
        "item": 9, # 9 = Lucerne / Alfalfa silage
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.lucerne_alfalfa_silage.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 9, # 9 = Lucerne / Alfalfa silage
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.lucerne_alfalfa_silage.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 9, # 9 = Lucerne / Alfalfa silage
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.lucerne_alfalfa_silage.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 9, # 9 = Lucerne / Alfalfa silage
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.lucerne_alfalfa_silage.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 4. Grass hay - on farm
    {
        "item": 4, # 4 = Grass hay
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.grass_hay.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 4, # 4 = Grass hay
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.grass_hay.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 4, # 4 = Grass hay
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.grass_hay.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 4, # 4 = Grass hay
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.grass_hay.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 11. Straw
    {
        "item": 11, # 11 = Straw
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.straw.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 11, # 11 = Straw
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.straw.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 11, # 11 = Straw
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.straw.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 11, # 11 = Straw
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.straw.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 13. Wheat
    {
        "item": 13, # 13 = Wheat
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.wheat.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 13, # 13 = Wheat
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.wheat.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 13, # 13 = Wheat
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.wheat.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 13, # 13 = Wheat
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.wheat.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 20. Other cereals
    {
        "item": 20, # 20 = Other cereals
        "region": "European Union + EFTA",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.other_cereals.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 20, # 20 = Other cereals
        "region": "European Union + EFTA",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.other_cereals.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 20, # 20 = Other cereals
        "region": "European Union + EFTA",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.other_cereals.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 20, # 20 = Other cereals
        "region": "European Union + EFTA",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.other_cereals.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 16. Oats
    {
        "item": 16, # 16 = Oats
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.oats.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 16, # 16 = Oats
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.oats.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 16, # 16 = Oats
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.oats.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 16, # 16 = Oats
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.oats.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },

    # 18. Rye
    {
        "item": 18, # 18 = Rye
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.rye.calf_dairy.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 18, # 18 = Rye
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.rye.heifer.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 18, # 18 = Rye
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.rye.cow_milk.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False
    },
    {        
        "item": 18, # 18 = Rye
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.rye.cow_dry.tonne"], 
            "unit": units["feed_weight"] 
            },
        "certified": False 
    },
    # 14. Barley
    {
        "item": 14,
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.barley.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 14,
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.barley.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 14,
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.barley.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 14,
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.barley.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 15. Maize / corn grain
    {
        "item": 15,
        "region": "European Union + EFTA",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.maize_grain.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 15,
        "region": "European Union + EFTA",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.maize_grain.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 15,
        "region": "European Union + EFTA",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.maize_grain.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 15,
        "region": "European Union + EFTA",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.maize_grain.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 31. Rapeseed cake/meal
    {
        "item": 31,
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.rapeseed_meal.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 31,
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.rapeseed_meal.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 31,
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.rapeseed_meal.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 31,
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.rapeseed_meal.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 27. Compound dairy feed: default (15% soy)
    {
        "item": 27,
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.compund_15_soy.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 27,
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.compund_15_soy.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 27,
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.compund_15_soy.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 27,
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.compund_15_soy.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 29. Compound dairy feed: low (8%) soy
    {
        "item": 29,
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.compund_8_soy.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 29,
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.compund_8_soy.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 29,
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.compund_8_soy.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 29,
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.compund_8_soy.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 28. Compound dairy feed: high (46%) soy
    {
        "item": 28,
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.compund_46_soy.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 28,
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.compund_46_soy.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 28,
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.compund_46_soy.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 28,
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.compund_46_soy.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
        },
    # 30. Soybean cake/meal
    {
        "item": 30,
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.soybean_meal.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 30,
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.soybean_meal.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 30,
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.soybean_meal.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 30,
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.soybean_meal.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 36. Soybean
    {
        "item": 36,
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.soybean.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 36,
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.soybean.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 36,
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.soybean.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 36,
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.soybean.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 42. Fodder beet
    {
        "item": 42,
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.fodder_beet.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 42,
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.fodder_beet.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 42,
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.fodder_beet.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 42,
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.fodder_beet.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 48. Sugar beet pulp
    {
        "item": 48,
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.sugar_beet_pulp.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 48,
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.sugar_beet_pulp.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 48,
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.sugar_beet_pulp.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 48,
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.sugar_beet_pulp.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 78. Molasses
    {
        "item": 78,
        "region": "Poland",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.molasses.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 78,
        "region": "Poland",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.molasses.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 78,
        "region": "Poland",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.molasses.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 78,
        "region": "Poland",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.molasses.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 23. dried_distillers_grains / Draff
    {
        "item": 23,
        "region": "European Union + EFTA",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.dried_distillers_grains.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 23,
        "region": "European Union + EFTA",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.dried_distillers_grains.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 23,
        "region": "European Union + EFTA",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.dried_distillers_grains.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 23,
        "region": "European Union + EFTA",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.dried_distillers_grains.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 24. Brewer’s grain
    {
        "item": 24,
        "region": "European Union + EFTA",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.brewers_grain.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 24,
        "region": "European Union + EFTA",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.brewers_grain.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 24,
        "region": "European Union + EFTA",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.brewers_grain.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 24,
        "region": "European Union + EFTA",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.brewers_grain.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 35. Other oil cake/meal
    {
        "item": 35,
        "region": "Global (world)",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.other_oil_meal.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 35,
        "region": "Global (world)",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.other_oil_meal.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 35,
        "region": "Global (world)",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.other_oil_meal.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 35,
        "region": "Global (world)",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.other_oil_meal.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 80. Mineral mix
    {
        "item": 80,
        "region": "European Union + EFTA",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.mineral_mix.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 80,
        "region": "European Union + EFTA",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.mineral_mix.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 80,
        "region": "European Union + EFTA",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.mineral_mix.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 80,
        "region": "European Union + EFTA",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.mineral_mix.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    # 75. Bypass fat
    {
        "item": 75,
        "region": "European Union + EFTA",
        "herd_section": "calf_dairy",
        "dry_matter": {
            "value": dff["feed.bypass_fat.calf_dairy.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 75,
        "region": "European Union + EFTA",
        "herd_section": "heifer",
        "dry_matter": {
            "value": dff["feed.bypass_fat.heifer.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 75,
        "region": "European Union + EFTA",
        "herd_section": "cow_milk",
        "dry_matter": {
            "value": dff["feed.bypass_fat.cow_milk.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
    {
        "item": 75,
        "region": "European Union + EFTA",
        "herd_section": "cow_dry",
        "dry_matter": {
            "value": dff["feed.bypass_fat.cow_dry.tonne"],
            "unit": units["feed_weight"]
        },
        "certified": False
    },
]

manure_inputs = []

herd_sections = ["calf_dairy", "heifer", "cow_milk", "cow_dry"]

for herd in herd_sections:
    manure_type = dff[f"manure_type.{herd}"] # names of columns
    
    if manure_type != "PIT STORAGE AND SOLID STORAGE":
        manure_inputs.extend([
            {"herd_section": herd, "type": 6, "allocation": 50},  # Pit Storage
            {"herd_section": herd, "type": 1, "allocation": 50},  # Solid Storage
        ])
    else:
        manure_inputs.append({
            "herd_section": herd,
            "type": manure_type,
            "allocation": 100
        })

    

dairy_input = {
    "farm": farm_input,
    "general": general_input,
    "milk_production": milk_production_input,
    "herd_sections": herd_sections_input,
    "grazing": grazing_input,
    "fertilizers": fertilizers_input,
    "feed_components": feed_components_input,
    "feed_additives": [], # leave empty as no additives
    "manure": manure_inputs,
    "direct_energy": [],
    "transport": []
}

KeyError: 'farm_id'